In [13]:
# import libaries
import pandas as pd
import numpy as np
from scipy import stats

In [127]:
df = pd.read_excel('Scenario Anonymised v2.xlsx') 

In [128]:
df.head(10)

,Flight Phase,Engine Serial Number,Operator,Time / seconds,Temperature / Degrees,Predicted Part Damage / mm,Actual Part Damage / mm,"Instructions:\nYou’ve been asked to implement the following engineering equation to determine the damage to a turbine blade during each flight from Engine Health Monitoring data. Please develop code to show how you might implement this and what your considerations might be in dealing with data:\n\nDamage = OPERATOR_FACTOR x 1.2E-11 x AVERAGE_TEMP_FOR_CLIMB_PHASE^2.88\n\nOperator factor - ABC = 1, DEF = 1.1\n\n1% Chance of data corruption\n\nPart Damage validation data is provided\n\nTemperature has 1% measurement uncertainty"
0,FlightPhaseEnum.taxiout,11101,ABC,0.000000,727.190125,NaN,NaN,NaN
1,FlightPhaseEnum.taxiout,11101,ABC,944.000000,727.190125,NaN,NaN,NaN
2,FlightPhaseEnum.takeoff,11101,ABC,949.000000,916.815519,NaN,NaN,NaN
3,FlightPhaseEnum.takeoff,11101,ABC,973.776886,921.807580,NaN,NaN,NaN
4,FlightPhaseEnum.takeoff,11101,ABC,980.000000,924.630174,NaN,NaN,NaN
5,FlightPhaseEnum.takeoff,11101,ABC,996.001000,935.805692,NaN,NaN,NaN
6,FlightPhaseEnum.takeoff,11101,ABC,1108.000000,935.805692,NaN,NaN,NaN
7,FlightPhaseEnum.climb,11101,ABC,1113.000000,882.757830,0.003407,0.003507,NaN
8,FlightPhaseEnum.climb,11101,ABC,1270.000000,902.854711,NaN,NaN,NaN
9,FlightPhaseEnum.climb,11101,ABC,1478.000000,909.088819,NaN,NaN,NaN


In [129]:
# select all columns except last three as there are not input features of the damage equation
df = df.iloc[: , :-3]

In [130]:
df

,Flight Phase,Engine Serial Number,Operator,Time / seconds,Temperature / Degrees
0,FlightPhaseEnum.taxiout,11101,ABC,0.000000,727.190125
1,FlightPhaseEnum.taxiout,11101,ABC,944.000000,727.190125
2,FlightPhaseEnum.takeoff,11101,ABC,949.000000,916.815519
3,FlightPhaseEnum.takeoff,11101,ABC,973.776886,921.807580
4,FlightPhaseEnum.takeoff,11101,ABC,980.000000,924.630174
...,...,...,...,...,...
90,FlightPhaseEnum.climb,11105,ABC,2161.000000,818.224388
91,FlightPhaseEnum.climb,11105,ABC,2507.551163,814.897915
92,FlightPhaseEnum.cruise,11105,ABC,2512.551163,771.389284
93,FlightPhaseEnum.cruise,11105,ABC,8276.000000,771.346432


In [131]:

cols = df.select_dtypes('number').columns  # limits to a (float), b (int) and e (timedelta)
df_sub = df.loc[:, cols]

In [132]:
 # drop rows with nan values
df.dropna(subset=cols, inplace=True)

In [133]:
df

,Flight Phase,Engine Serial Number,Operator,Time / seconds,Temperature / Degrees
0,FlightPhaseEnum.taxiout,11101,ABC,0.000000,727.190125
1,FlightPhaseEnum.taxiout,11101,ABC,944.000000,727.190125
2,FlightPhaseEnum.takeoff,11101,ABC,949.000000,916.815519
3,FlightPhaseEnum.takeoff,11101,ABC,973.776886,921.807580
4,FlightPhaseEnum.takeoff,11101,ABC,980.000000,924.630174
...,...,...,...,...,...
90,FlightPhaseEnum.climb,11105,ABC,2161.000000,818.224388
91,FlightPhaseEnum.climb,11105,ABC,2507.551163,814.897915
92,FlightPhaseEnum.cruise,11105,ABC,2512.551163,771.389284
93,FlightPhaseEnum.cruise,11105,ABC,8276.000000,771.346432


In [134]:
# group the categorical features to aggregate the mean of numerical features
df = df.groupby(['Flight Phase', 'Engine Serial Number','Operator']).agg({'Temperature / Degrees': ['mean'],'Time / seconds': ['mean']})

In [135]:
# create new columns of average temperature and average time of flight phase
df.columns = ['average_temp','average_flight_phase_time']
df = df.reset_index()
print(df)

               Flight Phase  Engine Serial Number Operator  average_temp  \
0     FlightPhaseEnum.climb                 11101      ABC    870.777901   
1     FlightPhaseEnum.climb                 11102      DEF    875.077487   
2     FlightPhaseEnum.climb                 11103      ABC    875.566523   
3     FlightPhaseEnum.climb                 11104      DEF    851.827002   
4     FlightPhaseEnum.climb                 11105      ABC    814.389669   
5    FlightPhaseEnum.cruise                 11101      ABC    789.233853   
6    FlightPhaseEnum.cruise                 11102      DEF    823.374563   
7    FlightPhaseEnum.cruise                 11103      ABC    794.542207   
8    FlightPhaseEnum.cruise                 11104      DEF    801.364220   
9    FlightPhaseEnum.cruise                 11105      ABC    771.360716   
10  FlightPhaseEnum.takeoff                 11101      ABC    926.972932   
11  FlightPhaseEnum.takeoff                 11102      DEF    937.350926   
12  FlightPh

In [139]:
def actual_part_damage(df):
   AVERAGE_TEMP_FOR_CLIMB_PHASE = df['average_temp']
   OPERATOR_FACTOR = {'ABC':1,'DEF':1.1}
   df['Op_factor'] = df['Operator'].map(OPERATOR_FACTOR)
   damage = df['Op_factor'] * 1.2e-11 * AVERAGE_TEMP_FOR_CLIMB_PHASE **2.88
   predicted_damage = damage - 1e-04

   df['actual_part_damage'] = damage
   df['predicted_part_damage'] = predicted_damage
   return df
   

      

In [140]:
# calling the function
df = actual_part_damage(df)

In [141]:
df

,Flight Phase,Engine Serial Number,Operator,average_temp,average_flight_phase_time,Op_factor,damage,actual_part_damage,predicted_part_damage
0,FlightPhaseEnum.climb,11101,ABC,870.777901,1909.954955,1.0,0.003517,0.003517,0.003417
1,FlightPhaseEnum.climb,11102,DEF,875.077487,1506.475399,1.1,0.003923,0.003923,0.003823
2,FlightPhaseEnum.climb,11103,ABC,875.566523,1320.418664,1.0,0.003573,0.003573,0.003473
3,FlightPhaseEnum.climb,11104,DEF,851.827002,2776.718405,1.1,0.003631,0.003631,0.003531
4,FlightPhaseEnum.climb,11105,ABC,814.389669,1717.394574,1.0,0.002900,0.002900,0.002800
5,FlightPhaseEnum.cruise,11101,ABC,789.233853,17103.727928,1.0,0.002649,0.002649,0.002549
6,FlightPhaseEnum.cruise,11102,DEF,823.374563,11675.696104,1.1,0.003292,0.003292,0.003192
7,FlightPhaseEnum.cruise,11103,ABC,794.542207,11489.942922,1.0,0.002701,0.002701,0.002601
8,FlightPhaseEnum.cruise,11104,DEF,801.364220,9649.116412,1.1,0.003045,0.003045,0.002945
9,FlightPhaseEnum.cruise,11105,ABC,771.360716,6385.517054,1.0,0.002480,0.002480,0.002380


In [ ]:
df.to_csv('damage_actual_predicted.csv')